In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install transformer-lens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_

In [ ]:
import torch
from torch import nn, Tensor
from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

# load model

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
model = HookedTransformer.from_pretrained("tiny-stories-1L-21M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/269M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-1L-21M into HookedTransformer


# load dataset

only get samples with specific concepts/words

In [ ]:
from datasets import load_dataset

In [ ]:
# Load the dataset in streaming mode
# dataset = load_dataset("Skylion007/openwebtext", split="train", streaming=True)
dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)

# Define the maximum sequence length for the model
max_length = 128

# Function to check if text contains the words "love" or "hate"
def contains_love_or_hate(text):
    # return "love" in text.lower() or "hate" in text.lower()
    return "she" in text.lower() or "her" in text.lower()

# Define a function to get tokens in batches with truncation and padding
def get_batch_tokens(dataset, tokenizer, batch_size=32, max_length=128):
    sequences = []
    love_hate_sequences = []
    other_sequences = []
    iterator = iter(dataset)  # Create an iterator from the streamed dataset

    # Separate sequences into those containing "love" or "hate" and those that do not
    for _ in range(batch_size * 2):  # Load more to ensure we get enough samples
        try:
            # Get the next example from the dataset
            example = next(iterator)
            text = example['text']
            if contains_love_or_hate(text):
                tokens = tokenizer.encode(text, max_length=max_length, truncation=True, padding="max_length", return_tensors='pt')
                love_hate_sequences.append(tokens)
            else:
                tokens = tokenizer.encode(text, max_length=max_length, truncation=True, padding="max_length", return_tensors='pt')
                other_sequences.append(tokens)
        except StopIteration:
            # If the dataset ends before reaching the required amount
            break

    # Ensure we have enough samples of each type
    # min_length = min(len(love_hate_sequences), len(other_sequences))
    # love_hate_sequences = love_hate_sequences[:min_length]
    # other_sequences = other_sequences[:min_length]

    others_len = batch_size - len(love_hate_sequences)
    other_sequences = other_sequences[:others_len]

    # Combine sequences to form the batch
    # sequences = love_hate_sequences[:batch_size//2] + other_sequences[:batch_size//2]
    sequences = love_hate_sequences + other_sequences

    # pdb.set_trace()

    if sequences:
        batch_tokens = torch.cat(sequences, dim=0).squeeze(1)
        return batch_tokens
    else:
        return None

# Get a batch of tokens
batch_tokens = get_batch_tokens(dataset, model.tokenizer, batch_size=100, max_length=max_length)
if batch_tokens is not None:
    print(batch_tokens.shape)
else:
    print("No data to load.")

Repo card metadata block was not found. Setting CardData to empty.


torch.Size([195, 128])


In [ ]:
import pickle

In [ ]:
with open('batch_tokens_sheHer.pkl', 'wb') as f:
    pickle.dump(batch_tokens, f)

In [ ]:
!cp batch_tokens_sheHer.pkl /content/drive/MyDrive/

In [ ]:
# check if saved
file_path = '/content/drive/MyDrive/batch_tokens_sheHer.pkl'
with open(file_path, 'rb') as f:
    batch_tokens = pickle.load(f)

# load sae features

In [ ]:
file_path = '/content/drive/MyDrive/fActs_ts_1L_21M_sheHer.pkl'
with open(file_path, 'rb') as f:
    feature_acts_model_A = pickle.load(f)

In [ ]:
file_path = '/content/drive/MyDrive/fActs_ts_2L_33M_sheHer.pkl'
with open(file_path, 'rb') as f:
    feature_acts_model_B = pickle.load(f)

# interpret top features by dataset examples

In [ ]:
def highest_activating_tokens(
    feature_acts,
    feature_idx: int,
    k: int = 10,  # num batch_seq samples
    batch_tokens=None
) -> Tuple[Int[Tensor, "k 2"], Float[Tensor, "k"]]:
    '''
    Returns the indices & values for the highest-activating tokens in the given batch of data.
    '''
    batch_size, seq_len = batch_tokens.shape

    # Get the top k largest activations for only targeted feature
    # need to flatten (batch,seq) into batch*seq first because it's ANY batch_seq, even if in same batch or same pos
    flattened_feature_acts = feature_acts[:, :, feature_idx].reshape(-1)

    top_acts_values, top_acts_indices = flattened_feature_acts.topk(k)
    # top_acts_values should be 1D
    # top_acts_indices should be also be 1D. Now, turn it back to 2D
    # Convert the indices into (batch, seq) indices
    top_acts_batch = top_acts_indices // seq_len
    top_acts_seq = top_acts_indices % seq_len

    return torch.stack([top_acts_batch, top_acts_seq], dim=-1), top_acts_values

In [ ]:
from rich import print as rprint
def display_top_sequences(top_acts_indices, top_acts_values, batch_tokens):
    s = ""
    for (batch_idx, seq_idx), value in zip(top_acts_indices, top_acts_values):
        # s += f'{batch_idx}\n'
        s += f'batchID: {batch_idx}, '
        # Get the sequence as a string (with some padding on either side of our sequence)
        seq_start = max(seq_idx - 5, 0)
        # seq_end = min(seq_idx + 5, all_tokens.shape[1])
        seq_end = min(seq_idx + 5, batch_tokens.shape[1])
        seq = ""
        # Loop over the sequence, adding each token to the string (highlighting the token with the large activations)
        for i in range(seq_start, seq_end):
            # new_str_token = model.to_single_str_token(tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            new_str_token = model.to_single_str_token(batch_tokens[batch_idx, i].item()).replace("\n", "\\n").replace("<|BOS|>", "|BOS|")
            if i == seq_idx:
                new_str_token = f"[bold u dark_orange]{new_str_token}[/]"
            seq += new_str_token
        # Print the sequence, and the activation value
        s += f'Act = {value:.2f}, Seq = "{seq}"\n'

    rprint(s)

In [ ]:
# 16244 in model A

# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

# top features in matching pair with model B
top_feats = [15028,  3580,  7634,  2208,  9768,  7110,  8427,   239,  3023,
        5859]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_B, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  15028


batchID: 138, Act = 0.35, Seq = "â€\n\nThey soon arrived at the market"
batchID: 79, Act = 0.34, Seq = " first you must promise to take care of your family"
batchID: 51, Act = 0.31, Seq = " lake.\n\nAs soon as the boy got"
batchID: 99, Act = 0.31, Seq = " John.\n\n"Of course," she said"
batchID: 91, Act = 0.30, Seq = " What do you need help with?â€\n"

Feature:  3580


batchID: 193, Act = 0.54, Seq = " little girl replied, "Well, you can't"
batchID: 187, Act = 0.37, Seq = "\nGrandma smiled and gave Charles some yarn to"
batchID: 113, Act = 0.33, Seq = " cry.\n\nMom gave John a hug,"
batchID: 88, Act = 0.32, Seq = "One day, Mary was sitting on the grass in"
batchID: 79, Act = 0.31, Seq = " flame, "Can you let me fly away so"

Feature:  7634


...

batchID: 118, Act = 1.58, Seq = "."\n\nThe two of them grabbed the dinosaur"
batchID: 193, Act = 1.55, Seq = " little girl replied, "Well, you can't"
batchID: 20, Act = 1.52, Seq = " He did not like to share his things with others"
batchID: 56, Act = 1.37, Seq = " mommy replied, "Oh, it's very"
batchID: 166, Act = 1.36, Seq = " turkey ran faster. Then something amazing happened. Jack"

Feature:  3023


batchID: 187, Act = 0.91, Seq = ""What are you doing grandma"
batchID: 141, Act = 0.85, Seq = ". They went up and down, side to"
batchID: 47, Act = 0.82, Seq = " had a big sign that said "Ice-C"
batchID: 10, Act = 0.82, Seq = " saw a big sign that said, "Fun Race"
batchID: 121, Act = 0.81, Seq = ", zipping up and down the wall!\n"

Feature:  5859


batchID: 50, Act = 0.31, Seq = " the button again and this time it"
batchID: 53, Act = 0.25, Seq = "But then, Emily noticed something strange. She pulled"
batchID: 120, Act = 0.22, Seq = ". Suddenly, Sue noticed something strange ��€"
batchID: 92, Act = 0.22, Seq = " so she pulled harder and harder. Suddenly, the"
batchID: 94, Act = 0.22, Seq = " the car, she noticed something strange. There was"

In [ ]:
# feature 0 in model A

# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

# top features in matching pair with model B
top_feats = [ 7219,  2090, 13160,  5793,  9053,  3751,  1351,  1786, 13994,
       10304]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_B, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  7219


batchID: 129, Act = 0.65, Seq = " and everyone began to sing happy birthday.\n\n"
batchID: 47, Act = 0.63, Seq = " had a big sign that said "Ice-C"
batchID: 10, Act = 0.57, Seq = " saw a big sign that said, "Fun Race"
batchID: 93, Act = 0.57, Seq = " animals who lived there. Every day he would go"
batchID: 161, Act = 0.56, Seq = " little boy asked the lady why her leg was so"

Feature:  2090


batchID: 53, Act = 0.95, Seq = "Mommy and Emily were playing"
batchID: 184, Act = 0.95, Seq = "Mommy and Daddy were eating"
batchID: 171, Act = 0.91, Seq = " Mia. "I like to whistle"
batchID: 117, Act = 0.85, Seq = " cheered as Mama and Papa gave the little girl a"
batchID: 133, Act = 0.79, Seq = " sad, for it no longer had any oranges left"

Feature:  13160


...

batchID: 55, Act = 1.11, Seq = " Lucy. She was three years old and she loved"
batchID: 92, Act = 1.10, Seq = "ola. She was three years old and she loved"
batchID: 158, Act = 1.06, Seq = " Jack. Jack was three years old. He wanted"
batchID: 129, Act = 1.06, Seq = " John. He was three years old and he was"
batchID: 159, Act = 1.05, Seq = " Jane. She was three years old and she was"

Feature:  13994


batchID: 102, Act = 1.51, Seq = " They were all looking just as scared as she felt"
batchID: 3, Act = 1.48, Seq = " better.\n\nAs time"
batchID: 190, Act = 1.46, Seq = " yummy, why don't you show it to"
batchID: 178, Act = 1.42, Seq = " the ocean. Her mom said to be careful,"
batchID: 173, Act = 1.34, Seq = ". Her mom smiled and said, "It's"

Feature:  10304


batchID: 20, Act = 0.91, Seq = ". He wanted to keep them all for himself."
batchID: 121, Act = 0.90, Seq = " \n\n"What's that noise?" asked"
batchID: 25, Act = 0.82, Seq = ". In the hole, there was a shiny coin"
batchID: 54, Act = 0.75, Seq = ". She wanted to know what a lawyer did and"
batchID: 84, Act = 0.75, Seq = " her dad, "What's he pointing at?""

1786 comes from above

In [ ]:
# feature 1 in model A

# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

# top features in matching pair with model B
top_feats = [  657,  4223,  9950, 12712,  4451,  7606,  5630,  2945, 12266,
        5642]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_B, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  657


batchID: 96, Act = 0.60, Seq = "my, Daddy! Look what we found!" "
batchID: 93, Act = 0.54, Seq = ", and said, "What is going on here"
batchID: 146, Act = 0.53, Seq = ", you must be gentle with it. Folding"
batchID: 53, Act = 0.52, Seq = "But then, Emily noticed something strange. She pulled"
batchID: 93, Act = 0.52, Seq = " day, the farmer noticed something strange. One of"

Feature:  4223


batchID: 3, Act = 0.97, Seq = " better.\n\nAs time"
batchID: 146, Act = 0.79, Seq = " Folding the phone out is a special thing you"
batchID: 100, Act = 0.78, Seq = " river had something to do with it.\n\n"
batchID: 18, Act = 0.75, Seq = " new friend did not know how to play. The"
batchID: 133, Act = 0.74, Seq = " birds, rabbits, and more all shared the oranges"

Feature:  9950


...

batchID: 187, Act = 1.59, Seq = " are you doing grandma?" asked Charles.\n\n"
batchID: 163, Act = 1.42, Seq = " "Be careful and remember to stay safe"
batchID: 8, Act = 1.40, Seq = " test your knee. Try to walk slowly and see"
batchID: 166, Act = 1.35, Seq = "s get the turkey!" said Jack. \n"
batchID: 187, Act = 1.33, Seq = ""What are you doing"

Feature:  12266


batchID: 58, Act = 1.65, Seq = " was only three years old but she was full of"
batchID: 9, Act = 1.55, Seq = "Do you want to go for a ride?"\n"
batchID: 51, Act = 1.41, Seq = " into the boat and go for a ride on"
batchID: 85, Act = 1.40, Seq = "He ran for hours and hours, never stopping for"
batchID: 39, Act = 1.33, Seq = " Binky had a long, waggy tail"

Feature:  5642


batchID: 54, Act = 2.03, Seq = ". So, Mia jumped right up to the lawyer"
batchID: 54, Act = 1.79, Seq = " a lawyer. Mia had never seen a lawyer before"
batchID: 76, Act = 1.51, Seq = " to a place he had never seen before. It"
batchID: 83, Act = 1.50, Seq = " the rabbit. He had never seen anything like this"
batchID: 121, Act = 1.49, Seq = " \n\n"What's that noise?" asked"

# B to A

In [ ]:
# feature 1786 in model B

# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

# top features in matching pair with model A
top_feats = [16251, 13152, 14923, 10373, 16144,  7364,  9425, 15399,  5912,
        3655]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_A, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  16251


batchID: 3, Act = 4.16, Seq = "Once upon a time,"
batchID: 1, Act = 4.16, Seq = "Once upon a time,"
batchID: 4, Act = 4.16, Seq = "Once upon a time,"
batchID: 6, Act = 4.16, Seq = "Once upon a time,"
batchID: 5, Act = 4.16, Seq = "Once upon a time,"

Feature:  13152


batchID: 83, Act = 0.63, Seq = "One summer morning, a little boy"
batchID: 76, Act = 0.47, Seq = "One day, Little Bear and his Mom"
batchID: 83, Act = 0.41, Seq = "One summer morning, a little"
batchID: 2, Act = 0.37, Seq = "One day, a little fish"
batchID: 0, Act = 0.37, Seq = "One day, a little girl"

Feature:  14923


...

batchID: 112, Act = 0.85, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 0.83, Seq = "Once upon a time,"
batchID: 1, Act = 0.83, Seq = "Once upon a time,"
batchID: 5, Act = 0.83, Seq = "Once upon a time,"
batchID: 3, Act = 0.83, Seq = "Once upon a time,"

Feature:  5912


batchID: 112, Act = 1.83, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 1.83, Seq = "Once upon a time,"
batchID: 1, Act = 1.83, Seq = "Once upon a time,"
batchID: 5, Act = 1.83, Seq = "Once upon a time,"
batchID: 3, Act = 1.83, Seq = "Once upon a time,"

Feature:  3655


batchID: 112, Act = 1.40, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 1.27, Seq = "Once upon a time,"
batchID: 1, Act = 1.27, Seq = "Once upon a time,"
batchID: 5, Act = 1.27, Seq = "Once upon a time,"
batchID: 3, Act = 1.27, Seq = "Once upon a time,"

In [ ]:
top_feats = [1786]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_B, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  1786


batchID: 55, Act = 1.11, Seq = " Lucy. She was three years old and she loved"
batchID: 92, Act = 1.10, Seq = "ola. She was three years old and she loved"
batchID: 158, Act = 1.06, Seq = " Jack. Jack was three years old. He wanted"
batchID: 129, Act = 1.06, Seq = " John. He was three years old and he was"
batchID: 159, Act = 1.05, Seq = " Jane. She was three years old and she was"

In [ ]:
# feature 3103 in model B

# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5

# top features in matching pair with model A
top_feats = [14923, 16251, 13152, 13166, 10373, 16144,  7364, 15399,  9425,
        5912]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_A, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  14923


batchID: 3, Act = 10.35, Seq = "Once upon a time, in a land"
batchID: 1, Act = 10.35, Seq = "Once upon a time, there was a"
batchID: 4, Act = 10.35, Seq = "Once upon a time, there was a"
batchID: 6, Act = 10.35, Seq = "Once upon a time, in a small"
batchID: 5, Act = 10.35, Seq = "Once upon a time, in a big"

Feature:  16251


batchID: 3, Act = 4.16, Seq = "Once upon a time,"
batchID: 1, Act = 4.16, Seq = "Once upon a time,"
batchID: 4, Act = 4.16, Seq = "Once upon a time,"
batchID: 6, Act = 4.16, Seq = "Once upon a time,"
batchID: 5, Act = 4.16, Seq = "Once upon a time,"

Feature:  13152


...

batchID: 112, Act = 0.85, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 0.83, Seq = "Once upon a time,"
batchID: 1, Act = 0.83, Seq = "Once upon a time,"
batchID: 5, Act = 0.83, Seq = "Once upon a time,"
batchID: 3, Act = 0.83, Seq = "Once upon a time,"

Feature:  9425


batchID: 112, Act = 4.09, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 3.66, Seq = "Once upon a time,"
batchID: 1, Act = 3.66, Seq = "Once upon a time,"
batchID: 5, Act = 3.66, Seq = "Once upon a time,"
batchID: 3, Act = 3.66, Seq = "Once upon a time,"

Feature:  5912


batchID: 112, Act = 1.83, Seq = " it outside.\n\nOnce she got to the"
batchID: 4, Act = 1.83, Seq = "Once upon a time,"
batchID: 1, Act = 1.83, Seq = "Once upon a time,"
batchID: 5, Act = 1.83, Seq = "Once upon a time,"
batchID: 3, Act = 1.83, Seq = "Once upon a time,"

In [ ]:
top_feats = [3103]

# for feature_idx in top_acts_indices[0, -1, :]:
for feature_idx in top_feats:
    # feature_idx = feature_idx.item()
    print('Feature: ', feature_idx)
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(feature_acts_model_B, feature_idx, samp_m, batch_tokens=batch_tokens)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, batch_tokens=batch_tokens)

Feature:  3103


batchID: 3, Act = 4.70, Seq = "Once upon a time,"
batchID: 1, Act = 4.70, Seq = "Once upon a time,"
batchID: 4, Act = 4.70, Seq = "Once upon a time,"
batchID: 6, Act = 4.70, Seq = "Once upon a time,"
batchID: 5, Act = 4.70, Seq = "Once upon a time,"